In [1]:
import micropip

await micropip.install("aiocoap[prettyprint]")
import aiocoap

ctx = await aiocoap.Context.create_client_context()

In [2]:
await micropip.install("ipywidgets")
import ipywidgets as widgets

slider = widgets.FloatSlider()
slider

FloatSlider(value=0.0)

In [3]:
from aiocoap import resource
import cbor2


class Position(resource.Resource):
    def __init__(self, widget):
        super().__init__()
        self.widget = widget

    async def render_get(self, request):
        # We should check some options here, or bug the aiocoap author
        # about better abstractions for typed resources
        return aiocoap.Message(
            content_format=60, payload=cbor2.dumps(self.widget.value)
        )

    async def render_put(self, request):
        self.widget.value = cbor2.loads(request.payload)
        return aiocoap.Message(code=aiocoap.CHANGED)


root = resource.Site()
root.add_resource(
    [".well-known", "core"], resource.WKCResource(root.get_resources_as_linkheader)
)
root.add_resource(["position"], Position(slider))
ctx.serversite = root

In [4]:
from aiocoap.resourcedirectory.client.register import Registerer

registerer = Registerer(
    ctx,
    "coaps+ws://rd.coap.amsuess.com",
    lt=60,
    registration_parameters={
        "ep": "jupyter",
        "proxy": "on",
    },
)